# How-to: Predict and update URLs for OPD data

#### TO-DO
- complete testing
- Might need requirements.txt and how to code for that, using % to install packages

## Env and packages

In [1]:
import prediction_funcs

## Example: Try to find a valid URL for a new year

**Additional spreadsheet fields:**
"Description", "source_url", "readme", "date_field",
"agency_field", "min_version", "query"

The `try_urls_years` function auto-fills the spreadsheet's "URL", "Year","last_coverage_check", "coverage_start", and "coverage_end".

In [4]:
url = "https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2016_csv/FeatureServer/0"
spreadsheet_fields = { #required fields
    "State": "CA",
    "SourceName": "Example PD",
    "Agency": "EXPD",
    "AgencyFull": "Example Police Department",
    "TableType": "Arrests",
    "DataType": "ArcGIS"
    # below required if "DataType" = "Carto" or "Ckan"
    # , dataset_id": "example-dataset-id" 
}

prediction_funcs.try_url_years(url, forward=True, verbose=True, spreadsheet_fields=spreadsheet_fields)

https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2017_csv/FeatureServer/0 already in spreadsheet and valid. Updated last_coverage_check.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2018_csv/FeatureServer/0 already in spreadsheet and valid. Updated last_coverage_check.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2019_csv/FeatureServer/0 already in spreadsheet and valid. Updated last_coverage_check.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2020_csv/FeatureServer/0 already in spreadsheet and valid. Updated last_coverage_check.
https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2021_csv/FeatureServer/0: not valid. Skipping.


## Example: Automatically update all outdated sources

In [ ]:
prediction_funcs.auto_update_sources(verbose=True)